In [6]:
import pandas as pd
import numpy as np

liste_prenoms_source = pd.read_csv("DS_PRENOMS_data.tmp.csv", sep=";")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [39]:
# Exploration
# display(liste_prenoms_source.tail())

display(liste_prenoms_source.isna().any())
display(liste_prenoms_source[liste_prenoms_source["PRENOM"].isna()])
# display(liste_prenoms_source.isna().sum())
# display(liste_prenoms_source.dtypes)
# display(liste_prenoms_source["GEO_OBJECT"].unique())
# display(liste_prenoms_source["GEO"].unique())

# display(liste_prenoms_source[liste_prenoms_source["PRENOM"].isna()])
# display(liste_prenoms_source[liste_prenoms_source["GEO_OBJECT"] == "REG"])

YOB            False
SEX            False
PRENOM          True
GEO            False
GEO_OBJECT     False
TIME_PERIOD    False
OBS_VALUE      False
dtype: bool

,YOB,SEX,PRENOM,GEO,GEO_OBJECT,TIME_PERIOD,OBS_VALUE
869559,XXXX,2,NaN,F,FRANCE,2023,"32,"
872645,2003,2,NaN,F,FRANCE,2023,"3,"


In [ ]:
# Nettoyage
df_clean = liste_prenoms_source.copy()
# df_clean.drop(columns=['TIME_PERIOD', "GEO_OBJECT"], inplace=True)

df_clean = df_clean[df_clean["GEO_OBJECT"] != "FRANCE"]
df_clean.loc[(df_clean["GEO"] == "94") & (df_clean["GEO_OBJECT"] == "REG"), 'GEO'] = "3A"

# display(df_clean[df_clean["GEO_OBJECT"] == "REG"])
# df_clean["GEO"] = df_clean["GEO"].astype(str).str.replace('F', '75')

df_clean["OBS_VALUE"] = df_clean["OBS_VALUE"].str.replace(',', '')
df_clean["OBS_VALUE"] = df_clean["OBS_VALUE"].astype("int")

df_clean["YOB"] = df_clean["YOB"].astype("int")
df_clean["GEO"] = df_clean["GEO"].astype("str")

df_clean.dropna(inplace=True)

df_clean["PRENOM"] = df_clean["PRENOM"].str.replace(r'_\d+', '', regex=True)
df_clean = df_clean[(df_clean["PRENOM"].str.lower() != "PRENOMS_RARES".lower()) & (df_clean["YOB"] != "XXXX")]

display(df_clean[df_clean["PRENOM"] == "NICOLAS"])
# df_clean[df_clean["PRENOM"].str.contains("_")]
# df_clean = df_clean[df_clean["YOB"] == "1978"]

df_clean.to_sql("prenom-clean.tmp.sql")

,YOB,SEX,PRENOM,GEO,GEO_OBJECT,TIME_PERIOD,OBS_VALUE
2962003,1904,1,NICOLAS,34,DEP,2023,3
2962005,1908,1,NICOLAS,69,DEP,2023,3
2962087,1954,1,NICOLAS,69,DEP,2023,9
2962088,1941,1,NICOLAS,75,DEP,2023,14
2962089,1941,1,NICOLAS,29,DEP,2023,5
...,...,...,...,...,...,...,...
4122076,1943,2,NICOLAS,972,DEP,2023,5
4122133,1945,2,NICOLAS,972,DEP,2023,4
4122195,1909,2,NICOLAS,972,DEP,2023,3
4122202,1944,2,NICOLAS,971,DEP,2023,3


In [42]:
# unique
df_unique = df_clean["PRENOM"].unique()

display(df_unique)
len(df_unique)

array(['CANDICE', 'CANDIDA', 'CANDY', ..., 'ZUBEYDE', 'ZUINA', 'ZOELY'],
      dtype=object)

15089

In [16]:
# Contient 10 000 premières entrées concernant les naissances d'enfants de sexe féminin dans toute la France
top_10000 = df_clean[(df_clean["SEX"] == 2)][:10000]
top_10000.reset_index(inplace=True)
top_10000

,index,YOB,SEX,PRENOM,GEO,GEO_OBJECT,OBS_VALUE
0,0,2010,2,CANDICE,25,DEP,13
1,1,2018,2,CANDICE,89,DEP,5
2,2,2015,2,CANDICE,28,DEP,3
3,3,2005,2,CANDICE,40,DEP,10
4,4,2010,2,CANDICE,41,DEP,4
...,...,...,...,...,...,...,...
9995,73798,1949,2,CARMEN,34,DEP,6
9996,73799,1944,2,CARMEN,02,DEP,3
9997,73800,1949,2,CARMEN,27,DEP,5
9998,73801,1951,2,CARMEN,31,DEP,11


In [44]:
df_prenom = df_clean.copy()
df_prenom[df_prenom["PRENOM"].str.lower() == "thomas"]

,YOB,SEX,PRENOM,GEO,GEO_OBJECT,TIME_PERIOD,OBS_VALUE
3349066,1927,1,THOMAS,69,DEP,2023,3
3349067,1912,1,THOMAS,13,DEP,2023,3
3349069,1908,1,THOMAS,972,DEP,2023,14
3349132,1901,1,THOMAS,66,DEP,2023,3
3349134,1916,1,THOMAS,34,DEP,2023,4
...,...,...,...,...,...,...,...
4452478,1948,2,THOMAS,972,DEP,2023,4
4452545,1941,2,THOMAS,972,DEP,2023,4
4452589,1931,2,THOMAS,971,DEP,2023,3
4452652,1961,2,THOMAS,971,DEP,2023,5


In [48]:
# Contient toutes les naissances de l'année de votre naissance dans la région de votre choix (plusieurs départements) 
data_regions = [{"number":"01","name":"Ain","region":"Auvergne-Rhône-Alpes"},{"number":"02","name":"Aisne","region":"Hauts-de-France"},{"number":"03","name":"Allier","region":"Auvergne-Rhône-Alpes"},{"number":"04","name":"Alpes-de-Haute-Provence","region":"Provence-Alpes-Côte d’Azur"},{"number":"05","name":"Hautes-Alpes","region":"Provence-Alpes-Côte d’Azur"},{"number":"06","name":"Alpes-Maritimes","region":"Provence-Alpes-Côte d’Azur"},{"number":"07","name":"Ardèche","region":"Auvergne-Rhône-Alpes"},{"number":"08","name":"Ardennes","region":"Grand Est"},{"number":"09","name":"Ariège","region":"Occitanie"},{"number":"10","name":"Aube","region":"Grand Est"},{"number":"11","name":"Aude","region":"Occitanie"},{"number":"12","name":"Aveyron","region":"Occitanie"},{"number":"13","name":"Bouches-du-Rhône","region":"Provence-Alpes-Côte d’Azur"},{"number":"14","name":"Calvados","region":"Normandie"},{"number":"15","name":"Cantal","region":"Auvergne-Rhône-Alpes"},{"number":"16","name":"Charente","region":"Nouvelle-Aquitaine"},{"number":"17","name":"Charente-Maritime","region":"Nouvelle-Aquitaine"},{"number":"18","name":"Cher","region":"Centre-Val de Loire"},{"number":"19","name":"Corrèze","region":"Nouvelle-Aquitaine"},{"number":"2A","name":"Corse-du-Sud","region":"Corse"},{"number":"2B","name":"Haute-Corse","region":"Corse"},{"number":"21","name":"Côte-d’Or","region":"Bourgogne-Franche-Comté"},{"number":"22","name":"Côtes-d’Armor","region":"Bretagne"},{"number":"23","name":"Creuse","region":"Nouvelle-Aquitaine"},{"number":"24","name":"Dordogne","region":"Nouvelle-Aquitaine"},{"number":"25","name":"Doubs","region":"Bourgogne-Franche-Comté"},{"number":"26","name":"Drôme","region":"Auvergne-Rhône-Alpes"},{"number":"27","name":"Eure","region":"Normandie"},{"number":"28","name":"Eure-et-Loir","region":"Centre-Val de Loire"},{"number":"29","name":"Finistère","region":"Bretagne"},{"number":"30","name":"Gard","region":"Occitanie"},{"number":"31","name":"Haute-Garonne","region":"Occitanie"},{"number":"32","name":"Gers","region":"Occitanie"},{"number":"33","name":"Gironde","region":"Nouvelle-Aquitaine"},{"number":"34","name":"Hérault","region":"Occitanie"},{"number":"35","name":"Ille-et-Vilaine","region":"Bretagne"},{"number":"36","name":"Indre","region":"Centre-Val de Loire"},{"number":"37","name":"Indre-et-Loire","region":"Centre-Val de Loire"},{"number":"38","name":"Isère","region":"Auvergne-Rhône-Alpes"},{"number":"39","name":"Jura","region":"Bourgogne-Franche-Comté"},{"number":"40","name":"Landes","region":"Nouvelle-Aquitaine"},{"number":"41","name":"Loir-et-Cher","region":"Centre-Val de Loire"},{"number":"42","name":"Loire","region":"Auvergne-Rhône-Alpes"},{"number":"43","name":"Haute-Loire","region":"Auvergne-Rhône-Alpes"},{"number":"44","name":"Loire-Atlantique","region":"Pays de la Loire"},{"number":"45","name":"Loiret","region":"Centre-Val de Loire"},{"number":"46","name":"Lot","region":"Occitanie"},{"number":"47","name":"Lot-et-Garonne","region":"Nouvelle-Aquitaine"},{"number":"48","name":"Lozère","region":"Occitanie"},{"number":"49","name":"Maine-et-Loire","region":"Pays de la Loire"},{"number":"50","name":"Manche","region":"Normandie"},{"number":"51","name":"Marne","region":"Grand Est"},{"number":"52","name":"Haute-Marne","region":"Grand Est"},{"number":"53","name":"Mayenne","region":"Pays de la Loire"},{"number":"54","name":"Meurthe-et-Moselle","region":"Grand Est"},{"number":"55","name":"Meuse","region":"Grand Est"},{"number":"56","name":"Morbihan","region":"Bretagne"},{"number":"57","name":"Moselle","region":"Grand Est"},{"number":"58","name":"Nièvre","region":"Bourgogne-Franche-Comté"},{"number":"59","name":"Nord","region":"Hauts-de-France"},{"number":"60","name":"Oise","region":"Hauts-de-France"},{"number":"61","name":"Orne","region":"Normandie"},{"number":"62","name":"Pas-de-Calais","region":"Hauts-de-France"},{"number":"63","name":"Puy-de-Dôme","region":"Auvergne-Rhône-Alpes"},{"number":"64","name":"Pyrénées-Atlantiques","region":"Nouvelle-Aquitaine"},{"number":"65","name":"Hautes-Pyrénées","region":"Occitanie"},{"number":"66","name":"Pyrénées-Orientales","region":"Occitanie"},{"number":"67","name":"Bas-Rhin","region":"Grand Est"},{"number":"68","name":"Haut-Rhin","region":"Grand Est"},{"number":"69","name":"Rhône","region":"Auvergne-Rhône-Alpes"},{"number":"70","name":"Haute-Saône","region":"Bourgogne-Franche-Comté"},{"number":"71","name":"Saône-et-Loire","region":"Bourgogne-Franche-Comté"},{"number":"72","name":"Sarthe","region":"Pays de la Loire"},{"number":"73","name":"Savoie","region":"Auvergne-Rhône-Alpes"},{"number":"74","name":"Haute-Savoie","region":"Auvergne-Rhône-Alpes"},{"number":"75","name":"Paris","region":"Ile-de-France"},{"number":"76","name":"Seine-Maritime","region":"Normandie"},{"number":"77","name":"Seine-et-Marne","region":"Ile-de-France"},{"number":"78","name":"Yvelines","region":"Ile-de-France"},{"number":"79","name":"Deux-Sèvres","region":"Nouvelle-Aquitaine"},{"number":"80","name":"Somme","region":"Hauts-de-France"},{"number":"81","name":"Tarn","region":"Occitanie"},{"number":"82","name":"Tarn-et-Garonne","region":"Occitanie"},{"number":"83","name":"Var","region":"Provence-Alpes-Côte d’Azur"},{"number":"84","name":"Vaucluse","region":"Provence-Alpes-Côte d’Azur"},{"number":"85","name":"Vendée","region":"Pays de la Loire"},{"number":"86","name":"Vienne","region":"Nouvelle-Aquitaine"},{"number":"87","name":"Haute-Vienne","region":"Nouvelle-Aquitaine"},{"number":"88","name":"Vosges","region":"Grand Est"},{"number":"89","name":"Yonne","region":"Bourgogne-Franche-Comté"},{"number":"90","name":"Territoire de Belfort","region":"Bourgogne-Franche-Comté"},{"number":"91","name":"Essonne","region":"Ile-de-France"},{"number":"92","name":"Hauts-de-Seine","region":"Ile-de-France"},{"number":"93","name":"Seine-Saint-Denis","region":"Ile-de-France"},{"number":"94","name":"Val-de-Marne","region":"Ile-de-France"},{"number":"95","name":"Val-d’Oise","region":"Ile-de-France"},{"number":"971","name":"Guadeloupe","region":"Guadeloupe"},{"number":"972","name":"Martinique","region":"Martinique"},{"number":"973","name":"Guyane","region":"Guyane"},{"number":"974","name":"La Réunion","region":"La Réunion"},{"number":"976","name":"Mayotte","region":"Mayotte"}]
filtered_arr = [p for p in data_regions if p["region"] == "Ile-de-France"]

df_idf = df_clean.copy()

def get_region(value):
    region_data = [p["region"] for p in data_regions if str(p["number"]) == str(value).strip()]

    if len(region_data) > 0:
        return region_data[0]
    return ""


df_idf = df_idf[(df_idf["GEO"].isin([p["number"] for p in filtered_arr])) & (df_idf["YOB"] == "2010")]
# df_idf["region"] = df_idf['GEO'].map(get_region)
df_idf["region"] = "Ile-de-France"

df_idf.reset_index(inplace=True, drop=True)
display(df_idf)


,YOB,SEX,PRENOM,GEO,GEO_OBJECT,TIME_PERIOD,OBS_VALUE,region
0,2010,2,CAPUCINE,94,DEP,2023,21,Ile-de-France
1,2010,2,CAPUCINE,91,DEP,2023,11,Ile-de-France
2,2010,2,CAPUCINE,92,DEP,2023,38,Ile-de-France
3,2010,2,CAPUCINE,75,DEP,2023,71,Ile-de-France
4,2010,2,CAPUCINE,77,DEP,2023,5,Ile-de-France
...,...,...,...,...,...,...,...,...
834,2010,1,ABDERAHMANE,95,DEP,2023,3,Ile-de-France
835,2010,1,ABDOU,93,DEP,2023,7,Ile-de-France
836,2010,1,ABDERAHMANE,93,DEP,2023,4,Ile-de-France
837,2010,1,ABDELRAHMAN,75,DEP,2023,3,Ile-de-France


In [61]:
top_n_derniers_prenoms = (
    df_clean[df_clean["SEX"] == 2].groupby(['PRENOM', 'YOB'], as_index=False)["OBS_VALUE"].sum()
)
# display(top_n_derniers_prenoms)


res = (top_n_derniers_prenoms.groupby(['YOB'], as_index=False).apply(lambda grp: grp.nlargest(5, "OBS_VALUE")))
res.reset_index(drop=True, inplace=True)
res[res["YOB"] == "2023"]

,PRENOM,YOB,OBS_VALUE
0,AADHIRA,2020,8
1,AADHIRA,2021,6
2,AADHIRA,2022,8
3,AADHIRA,2023,5
4,AALEYAH,2018,6
...,...,...,...
348617,ZYNEB,2019,7
348618,ZYNEB,2020,8
348619,ZYNEB,2021,6
348620,ZYNEB,2022,4


,PRENOM,YOB,OBS_VALUE
615,CHARLIE,2023,3343
616,LOUISE,2023,3186
617,CHLOE,2023,3180
618,AMBRE,2023,3168
619,ELENA,2023,3143


In [66]:
# Quel est le prénom masculin et féminin le plus populaire de l'année 1995 dans la France entière (Territoire d'Outre-Mer + Metropole)
annee = 1995
grouped_births_1995 = (
    df_clean[df_clean["YOB"] == annee].groupby(['PRENOM', "SEX"], as_index=False)["OBS_VALUE"].sum()
)

top_births = (grouped_births_1995.groupby(['SEX'], as_index=False).apply(lambda grp: grp.nlargest(1, "OBS_VALUE")))
top_births.reset_index(drop=True, inplace=True)
total_births = df_clean[df_clean["YOB"] == annee].groupby(["SEX"], as_index=False)["OBS_VALUE"].sum()
# display(top_births)

display(top_births[(top_births["SEX"] == 1)]["OBS_VALUE"] / total_births[(total_births["SEX"] == 1)]["OBS_VALUE"] * 100)
display(top_births[(top_births["SEX"] == 2)]["OBS_VALUE"] / total_births[(total_births["SEX"] == 2)]["OBS_VALUE"] * 100)
# # part_garcons = 

0    2.795144
Name: OBS_VALUE, dtype: float64

1    3.348989
Name: OBS_VALUE, dtype: float64

In [68]:
# Quel est le prénom masculin et féminin le plus populaire de l'année 1995 dans la France entière (Territoire d'Outre-Mer + Metropole)
annee = 1905
grouped_births_1995 = (
    df_clean[df_clean["YOB"] == annee].groupby(['PRENOM', "SEX"], as_index=False)["OBS_VALUE"].sum()
)

top_births = (grouped_births_1995.groupby(['SEX'], as_index=False).apply(lambda grp: grp.nlargest(1, "OBS_VALUE")))
top_births.reset_index(drop=True, inplace=True)
display(top_births)
total_births = df_clean[df_clean["YOB"] == annee].groupby(["SEX"], as_index=False)["OBS_VALUE"].sum()
display(total_births)

display(top_births[(top_births["SEX"] == 1)]["OBS_VALUE"] / total_births[(total_births["SEX"] == 1)]["OBS_VALUE"] * 100)
display(top_births[(top_births["SEX"] == 2)]["OBS_VALUE"] / total_births[(total_births["SEX"] == 2)]["OBS_VALUE"] * 100)
# part_garcons = 

,PRENOM,SEX,OBS_VALUE
0,JEAN,1,17998
1,MARIE,2,48981


,SEX,OBS_VALUE
0,1,210945
1,2,234632


0    8.532082
Name: OBS_VALUE, dtype: float64

1    20.875669
Name: OBS_VALUE, dtype: float64

In [ ]:
# Pour un prénom choisi, calculer le nombre de naissances par an pour ce prénom et ordonnez par ordre croissant

df = df_clean.copy()
df = df[df["SEX"] == 2]

department_mean_birth = (
    df[df["PRENOM"] == "MARIE"].groupby(["YOB"], as_index=False)["OBS_VALUE"].sum()
)

total_births = (
    df.groupby(["YOB"], as_index=False)["OBS_VALUE"].sum()
)

department_mean_birth["percentage"] = department_mean_birth["OBS_VALUE"] / total_births["OBS_VALUE"] * 100

display(df)
display(department_mean_birth.sort_values(by=['OBS_VALUE'], ascending=False))

,YOB,SEX,PRENOM,GEO,OBS_VALUE
0,2010,2,CANDICE,25,13
1,2018,2,CANDICE,89,5
2,2015,2,CANDICE,28,3
3,2005,2,CANDICE,40,10
4,2010,2,CANDICE,41,4
...,...,...,...,...,...
1047172,1938,2,EMILIENNE,73,4
1047173,1937,2,EMILIENNE,65,7
1047343,1987,2,EMILY,38,3
1047344,2012,2,EMILY,06,7


,YOB,OBS_VALUE,percentage
1,1901,52150,19.092354
2,1902,51857,18.701292
3,1903,50424,18.181818
4,1904,50131,17.858135
5,1905,48981,17.246831
...,...,...,...
119,2019,752,0.208014
120,2020,658,0.187723
121,2021,639,0.180285
123,2023,597,0.188062


In [76]:
# Calculez le nombre de prénoms uniques par département toute années confondues

# display(
# df_clean.groupby(["GEO", "SEX"])["PRENOM"].nunique().reset_index()
# )

# df_clean[df_clean["GEO"] == "93"]["PRENOM"].unique()

hypo = df_clean[df_clean["SEX"] == 2].groupby(["PRENOM"], as_index=False)["OBS_VALUE"].sum()

display(
    hypo[hypo["PRENOM"].str.endswith("INE")]
)
hypo[hypo["PRENOM"].str.endswith("INE")].to_csv("test.tmp.csv")

,PRENOM,OBS_VALUE
16,ABELINE,6
46,ADALINE,142
59,ADELINE,49767
60,ADELPHINE,49
67,ADINE,3
...,...,...
8458,YSALINE,369
8479,YVELINE,7464
8542,ZELINE,3
8554,ZEPHIRINE,12


In [146]:
# Contient tous les prénoms epicenes
df_filter = (
    df_clean
        .groupby(['PRENOM', 'SEX'], as_index=False)
        .sum()
)

df_epicene = df_filter[df_filter.duplicated('PRENOM', keep=False)]


# df_epicene['diff'] = df_epicene.groupby(["PRENOM"])['OBS_VALUE'].apply(lambda x: np.diff(x).item())
# df_epicene['diff'] = df_epicene.groupby(["PRENOM"])['OBS_VALUE'].diff().fillna(0)
# df_epicene["diff"] = df_epicene[df_epicene["SEX"] == 1]["OBS_VALUE"] - 5

df_epicene

df_epicene_diff = df_epicene.groupby(["PRENOM"])['OBS_VALUE'].apply(lambda x: np.diff(x).item()).reset_index()
df_epicene_diff["OBS_VALUE"] = df_epicene_diff["OBS_VALUE"].abs()

display(df_epicene_diff)

res = df_epicene.merge(df_epicene_diff, left_on='PRENOM', right_on='PRENOM')
res.rename(columns={"OBS_VALUE_x": "OBS_VALUE", "OBS_VALUE_y": "diff"}, inplace=True)

display(
    res
)

,PRENOM,OBS_VALUE
0,AARON,70855
1,ABDON,939
2,ABEL,32696
3,ABI,77
4,ABIGAEL,2864
...,...,...
2138,ZINE,325
2139,ZION,389
2140,ZIYA,8
2141,ZOE,71389


,PRENOM,SEX,OBS_VALUE,diff
0,AARON,1,70926,70855
1,AARON,2,71,70855
2,ABDON,1,988,939
3,ABDON,2,49,939
4,ABEL,1,32840,32696
...,...,...,...,...
4281,ZIYA,2,70,8
4282,ZOE,1,323,71389
4283,ZOE,2,71712,71389
4284,ZOLA,1,139,688
